This code will use the reversed engineered tickers [here](https://www.kaggle.com/datasets/zyy2016/ubiquant-stock) to grab the stock prices.

In [1]:
import pandas as pd
import numpy as np
import gc
import yfinance as yf

Reading in the data as a Parquet file.

In [2]:
df = pd.read_parquet('parquet/train_low_mem.parquet')

CPU times: user 2.94 s, sys: 4.66 s, total: 7.6 s
Wall time: 6.79 s


Now we read in the mapping between investment_id and their ticker.

In [4]:
dictionary = pd.read_csv('RE Stocks/map_info.csv')

In [5]:
dictionary.shape

(3505, 11)

I load in the dictionary between `investment_id` and the ticker. For the Shanghai stocks, the last letter is 'H', which Yahoo Finance cannot find, but if you replace it with 'S', it will be found correctly.

In [6]:
mapping = {dictionary['investment_id'].iloc[i]:dictionary['Ticker'].iloc[i] for i in range(0,len(dictionary))}
for i in mapping.keys():
    if mapping[i][8] == 'H':
        mapping[i] = mapping[i][0:8]+'S'

Creating a dictionary between `time_id` to the actual date. I just use `000005.SZ` as a placeholder ticker.

In [7]:
temp = yf.Ticker('000005.SZ')
hist = temp.history(start='2014-01-01', end="2018-12-31")
timemap = hist.index

Testing out the yfinance API.

In [17]:
stock = yf.download(mapping[0], start='2014-01-01', end='2018-12-31')
price = stock

[*********************100%***********************]  1 of 1 completed


Creating an array of array of min timeid and max timeid.

In [18]:
price

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-20,11.926666,11.926666,11.926666,11.926666,11.580482,5194
2017-01-23,13.120000,13.120000,13.120000,13.120000,12.739176,3309
2017-01-24,14.433333,14.433333,14.433333,14.433333,14.014390,8400
2017-01-25,15.880000,15.880000,15.880000,15.880000,15.419065,35589
2017-01-26,17.466665,17.466665,17.466665,17.466665,16.959673,38019
...,...,...,...,...,...,...
2018-12-24,28.280001,29.000000,27.629999,28.790001,28.160536,1101176
2018-12-25,28.860001,28.930000,27.959999,28.799999,28.170317,1024195
2018-12-26,28.600000,29.920000,27.900000,29.920000,29.265829,1482726


In [19]:
timerange = [757, 1219]

In [20]:
price.loc[price.index>=timemap[757]]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-02-13,29.200001,31.586666,29.000000,31.386665,30.475630,24687981
2017-02-14,30.666666,31.246666,30.200001,30.700001,29.808899,14951134
2017-02-15,30.773333,33.313332,30.666666,31.353333,30.443266,22258396
2017-02-16,30.666666,30.706667,29.006666,29.600000,28.740826,15460311
2017-02-17,29.133333,29.786667,27.200001,27.379999,26.585260,13167546
...,...,...,...,...,...,...
2018-12-24,28.280001,29.000000,27.629999,28.790001,28.160536,1101176
2018-12-25,28.860001,28.930000,27.959999,28.799999,28.170317,1024195
2018-12-26,28.600000,29.920000,27.900000,29.920000,29.265829,1482726


In [21]:
price = yf.download(mapping[0], start='2014-01-01', end='2018-12-31') 
k = timerange[1] - timerange[0] +1
temparr = pd.DataFrame({'Price':price.loc[(price.index>=timemap[timerange[i][0]]) & (price.index<=timemap[timerange[i][1]])].Close}).reset_index(drop=True)
if len(temparr)<k:
    temparr2 = pd.DataFrame({'Price': [0]*(k-len(temparr))})
    temparr = pd.concat([temparr2, temparr])
tempdf = pd.DataFrame({'time_id': range(timerange[i][0], timerange[i][1]+1), 
                               'investment_id':[i]*k, 
                               'Price':temparr.Price })
pricelistdf = pd.concat([pricelistdf, tempdf])# = price.loc[price.index==timemap[k]].Close
print(i,"ended")

[*********************100%***********************]  1 of 1 completed


TypeError: list indices must be integers or slices, not numpy.float64

In [22]:
timerange = []
for i in range(0,3774):
    print("Starting ",i)
    timerange.append([df.loc[df['investment_id'] == i]['time_id'].min(), df.loc[df['investment_id'] == i]['time_id'].max()])
    print(i,' finished')

Starting  0
0  finished
Starting  1
1  finished
Starting  2
2  finished
Starting  3
3  finished
Starting  4
4  finished
Starting  5
5  finished
Starting  6
6  finished
Starting  7
7  finished
Starting  8
8  finished
Starting  9
9  finished
Starting  10
10  finished
Starting  11
11  finished
Starting  12
12  finished
Starting  13
13  finished
Starting  14
14  finished
Starting  15
15  finished
Starting  16
16  finished
Starting  17
17  finished
Starting  18
18  finished
Starting  19
19  finished
Starting  20
20  finished
Starting  21
21  finished
Starting  22
22  finished
Starting  23
23  finished
Starting  24
24  finished
Starting  25
25  finished
Starting  26
26  finished
Starting  27
27  finished
Starting  28
28  finished
Starting  29
29  finished
Starting  30
30  finished
Starting  31
31  finished
Starting  32
32  finished
Starting  33
33  finished
Starting  34
34  finished
Starting  35
35  finished
Starting  36
36  finished
Starting  37
37  finished
Starting  38
38  finished
Starti

305  finished
Starting  306
306  finished
Starting  307
307  finished
Starting  308
308  finished
Starting  309
309  finished
Starting  310
310  finished
Starting  311
311  finished
Starting  312
312  finished
Starting  313
313  finished
Starting  314
314  finished
Starting  315
315  finished
Starting  316
316  finished
Starting  317
317  finished
Starting  318
318  finished
Starting  319
319  finished
Starting  320
320  finished
Starting  321
321  finished
Starting  322
322  finished
Starting  323
323  finished
Starting  324
324  finished
Starting  325
325  finished
Starting  326
326  finished
Starting  327
327  finished
Starting  328
328  finished
Starting  329
329  finished
Starting  330
330  finished
Starting  331
331  finished
Starting  332
332  finished
Starting  333
333  finished
Starting  334
334  finished
Starting  335
335  finished
Starting  336
336  finished
Starting  337
337  finished
Starting  338
338  finished
Starting  339
339  finished
Starting  340
340  finished
Starti

615  finished
Starting  616
616  finished
Starting  617
617  finished
Starting  618
618  finished
Starting  619
619  finished
Starting  620
620  finished
Starting  621
621  finished
Starting  622
622  finished
Starting  623
623  finished
Starting  624
624  finished
Starting  625
625  finished
Starting  626
626  finished
Starting  627
627  finished
Starting  628
628  finished
Starting  629
629  finished
Starting  630
630  finished
Starting  631
631  finished
Starting  632
632  finished
Starting  633
633  finished
Starting  634
634  finished
Starting  635
635  finished
Starting  636
636  finished
Starting  637
637  finished
Starting  638
638  finished
Starting  639
639  finished
Starting  640
640  finished
Starting  641
641  finished
Starting  642
642  finished
Starting  643
643  finished
Starting  644
644  finished
Starting  645
645  finished
Starting  646
646  finished
Starting  647
647  finished
Starting  648
648  finished
Starting  649
649  finished
Starting  650
650  finished
Starti

918  finished
Starting  919
919  finished
Starting  920
920  finished
Starting  921
921  finished
Starting  922
922  finished
Starting  923
923  finished
Starting  924
924  finished
Starting  925
925  finished
Starting  926
926  finished
Starting  927
927  finished
Starting  928
928  finished
Starting  929
929  finished
Starting  930
930  finished
Starting  931
931  finished
Starting  932
932  finished
Starting  933
933  finished
Starting  934
934  finished
Starting  935
935  finished
Starting  936
936  finished
Starting  937
937  finished
Starting  938
938  finished
Starting  939
939  finished
Starting  940
940  finished
Starting  941
941  finished
Starting  942
942  finished
Starting  943
943  finished
Starting  944
944  finished
Starting  945
945  finished
Starting  946
946  finished
Starting  947
947  finished
Starting  948
948  finished
Starting  949
949  finished
Starting  950
950  finished
Starting  951
951  finished
Starting  952
952  finished
Starting  953
953  finished
Starti

1218  finished
Starting  1219
1219  finished
Starting  1220
1220  finished
Starting  1221
1221  finished
Starting  1222
1222  finished
Starting  1223
1223  finished
Starting  1224
1224  finished
Starting  1225
1225  finished
Starting  1226
1226  finished
Starting  1227
1227  finished
Starting  1228
1228  finished
Starting  1229
1229  finished
Starting  1230
1230  finished
Starting  1231
1231  finished
Starting  1232
1232  finished
Starting  1233
1233  finished
Starting  1234
1234  finished
Starting  1235
1235  finished
Starting  1236
1236  finished
Starting  1237
1237  finished
Starting  1238
1238  finished
Starting  1239
1239  finished
Starting  1240
1240  finished
Starting  1241
1241  finished
Starting  1242
1242  finished
Starting  1243
1243  finished
Starting  1244
1244  finished
Starting  1245
1245  finished
Starting  1246
1246  finished
Starting  1247
1247  finished
Starting  1248
1248  finished
Starting  1249
1249  finished
Starting  1250
1250  finished
Starting  1251
1251  fini

...

2654  finished
Starting  2655
2655  finished
Starting  2656
2656  finished
Starting  2657
2657  finished
Starting  2658
2658  finished
Starting  2659
2659  finished
Starting  2660
2660  finished
Starting  2661
2661  finished
Starting  2662
2662  finished
Starting  2663
2663  finished
Starting  2664
2664  finished
Starting  2665
2665  finished
Starting  2666
2666  finished
Starting  2667
2667  finished
Starting  2668
2668  finished
Starting  2669
2669  finished
Starting  2670
2670  finished
Starting  2671
2671  finished
Starting  2672
2672  finished
Starting  2673
2673  finished
Starting  2674
2674  finished
Starting  2675
2675  finished
Starting  2676
2676  finished
Starting  2677
2677  finished
Starting  2678
2678  finished
Starting  2679
2679  finished
Starting  2680
2680  finished
Starting  2681
2681  finished
Starting  2682
2682  finished
Starting  2683
2683  finished
Starting  2684
2684  finished
Starting  2685
2685  finished
Starting  2686
2686  finished
Starting  2687
2687  fini

2931  finished
Starting  2932
2932  finished
Starting  2933
2933  finished
Starting  2934
2934  finished
Starting  2935
2935  finished
Starting  2936
2936  finished
Starting  2937
2937  finished
Starting  2938
2938  finished
Starting  2939
2939  finished
Starting  2940
2940  finished
Starting  2941
2941  finished
Starting  2942
2942  finished
Starting  2943
2943  finished
Starting  2944
2944  finished
Starting  2945
2945  finished
Starting  2946
2946  finished
Starting  2947
2947  finished
Starting  2948
2948  finished
Starting  2949
2949  finished
Starting  2950
2950  finished
Starting  2951
2951  finished
Starting  2952
2952  finished
Starting  2953
2953  finished
Starting  2954
2954  finished
Starting  2955
2955  finished
Starting  2956
2956  finished
Starting  2957
2957  finished
Starting  2958
2958  finished
Starting  2959
2959  finished
Starting  2960
2960  finished
Starting  2961
2961  finished
Starting  2962
2962  finished
Starting  2963
2963  finished
Starting  2964
2964  fini

3205  finished
Starting  3206
3206  finished
Starting  3207
3207  finished
Starting  3208
3208  finished
Starting  3209
3209  finished
Starting  3210
3210  finished
Starting  3211
3211  finished
Starting  3212
3212  finished
Starting  3213
3213  finished
Starting  3214
3214  finished
Starting  3215
3215  finished
Starting  3216
3216  finished
Starting  3217
3217  finished
Starting  3218
3218  finished
Starting  3219
3219  finished
Starting  3220
3220  finished
Starting  3221
3221  finished
Starting  3222
3222  finished
Starting  3223
3223  finished
Starting  3224
3224  finished
Starting  3225
3225  finished
Starting  3226
3226  finished
Starting  3227
3227  finished
Starting  3228
3228  finished
Starting  3229
3229  finished
Starting  3230
3230  finished
Starting  3231
3231  finished
Starting  3232
3232  finished
Starting  3233
3233  finished
Starting  3234
3234  finished
Starting  3235
3235  finished
Starting  3236
3236  finished
Starting  3237
3237  finished
Starting  3238
3238  fini

3485  finished
Starting  3486
3486  finished
Starting  3487
3487  finished
Starting  3488
3488  finished
Starting  3489
3489  finished
Starting  3490
3490  finished
Starting  3491
3491  finished
Starting  3492
3492  finished
Starting  3493
3493  finished
Starting  3494
3494  finished
Starting  3495
3495  finished
Starting  3496
3496  finished
Starting  3497
3497  finished
Starting  3498
3498  finished
Starting  3499
3499  finished
Starting  3500
3500  finished
Starting  3501
3501  finished
Starting  3502
3502  finished
Starting  3503
3503  finished
Starting  3504
3504  finished
Starting  3505
3505  finished
Starting  3506
3506  finished
Starting  3507
3507  finished
Starting  3508
3508  finished
Starting  3509
3509  finished
Starting  3510
3510  finished
Starting  3511
3511  finished
Starting  3512
3512  finished
Starting  3513
3513  finished
Starting  3514
3514  finished
Starting  3515
3515  finished
Starting  3516
3516  finished
Starting  3517
3517  finished
Starting  3518
3518  fini

3765  finished
Starting  3766
3766  finished
Starting  3767
3767  finished
Starting  3768
3768  finished
Starting  3769
3769  finished
Starting  3770
3770  finished
Starting  3771
3771  finished
Starting  3772
3772  finished
Starting  3773
3773  finished


In [23]:
pricelistdf = pd.DataFrame({'time_id':[], 'investment_id':[], 'Price':[]}, )
for i in range(0,3774):
    print('Starting',i)
    if i in mapping.keys():
        price = yf.download(mapping[i], start='2014-01-01', end='2018-12-31')
        k = timerange[i][1] - timerange[i][0] +1
        temparr = pd.DataFrame({'Price':price.loc[(price.index>=timemap[timerange[i][0]]) & (price.index<=timemap[timerange[i][1]])].Close}).reset_index(drop=True)
        if len(temparr)<k:
            temparr2 = pd.DataFrame({'Price': [0]*(k-len(temparr))})
            temparr = pd.concat([temparr2, temparr])
        tempdf = pd.DataFrame({'time_id': range(timerange[i][0], timerange[i][1]+1), 
                               'investment_id':[i]*k, 
                               'Price':temparr.Price })
        pricelistdf = pd.concat([pricelistdf, tempdf])# = price.loc[price.index==timemap[k]].Close
    print(i,"ended")

Starting 0
[*********************100%***********************]  1 of 1 completed
0 ended
Starting 1
[*********************100%***********************]  1 of 1 completed
1 ended
Starting 2
[*********************100%***********************]  1 of 1 completed
2 ended
Starting 3
[*********************100%***********************]  1 of 1 completed
3 ended
Starting 4
[*********************100%***********************]  1 of 1 completed
4 ended
Starting 5
5 ended
Starting 6
[*********************100%***********************]  1 of 1 completed
6 ended
Starting 7
[*********************100%***********************]  1 of 1 completed
7 ended
Starting 8
[*********************100%***********************]  1 of 1 completed
8 ended
Starting 9
[*********************100%***********************]  1 of 1 completed
9 ended
Starting 10
[*********************100%***********************]  1 of 1 completed
10 ended
Starting 11
[*********************100%***********************]  1 of 1 completed
11 ended
Starting 

[*********************100%***********************]  1 of 1 completed
193 ended
Starting 194
[*********************100%***********************]  1 of 1 completed
194 ended
Starting 195
[*********************100%***********************]  1 of 1 completed
195 ended
Starting 196
[*********************100%***********************]  1 of 1 completed
196 ended
Starting 197
[*********************100%***********************]  1 of 1 completed
197 ended
Starting 198
[*********************100%***********************]  1 of 1 completed
198 ended
Starting 199
[*********************100%***********************]  1 of 1 completed
199 ended
Starting 200
[*********************100%***********************]  1 of 1 completed
200 ended
Starting 201
[*********************100%***********************]  1 of 1 completed
201 ended
Starting 202
[*********************100%***********************]  1 of 1 completed
202 ended
Starting 203
[*********************100%***********************]  1 of 1 completed
203 ended
S

[*********************100%***********************]  1 of 1 completed
382 ended
Starting 383
[*********************100%***********************]  1 of 1 completed
383 ended
Starting 384
[*********************100%***********************]  1 of 1 completed
384 ended
Starting 385
[*********************100%***********************]  1 of 1 completed
385 ended
Starting 386
[*********************100%***********************]  1 of 1 completed
386 ended
Starting 387
[*********************100%***********************]  1 of 1 completed
387 ended
Starting 388
[*********************100%***********************]  1 of 1 completed
388 ended
Starting 389
[*********************100%***********************]  1 of 1 completed
389 ended
Starting 390
[*********************100%***********************]  1 of 1 completed
390 ended
Starting 391
[*********************100%***********************]  1 of 1 completed
391 ended
Starting 392
[*********************100%***********************]  1 of 1 completed
392 ended
S

[*********************100%***********************]  1 of 1 completed
476 ended
Starting 477
477 ended
Starting 478
[*********************100%***********************]  1 of 1 completed
478 ended
Starting 479
[*********************100%***********************]  1 of 1 completed
479 ended
Starting 480
[*********************100%***********************]  1 of 1 completed
480 ended
Starting 481
[*********************100%***********************]  1 of 1 completed
481 ended
Starting 482
[*********************100%***********************]  1 of 1 completed
482 ended
Starting 483
[*********************100%***********************]  1 of 1 completed
483 ended
Starting 484
[*********************100%***********************]  1 of 1 completed
484 ended
Starting 485
[*********************100%***********************]  1 of 1 completed
485 ended
Starting 486
[*********************100%***********************]  1 of 1 completed
486 ended
Starting 487
[*********************100%***********************]  1 of 

[*********************100%***********************]  1 of 1 completed
665 ended
Starting 666
[*********************100%***********************]  1 of 1 completed
666 ended
Starting 667
[*********************100%***********************]  1 of 1 completed
667 ended
Starting 668
[*********************100%***********************]  1 of 1 completed
668 ended
Starting 669
[*********************100%***********************]  1 of 1 completed
669 ended
Starting 670
[*********************100%***********************]  1 of 1 completed
670 ended
Starting 671
[*********************100%***********************]  1 of 1 completed
671 ended
Starting 672
[*********************100%***********************]  1 of 1 completed
672 ended
Starting 673
[*********************100%***********************]  1 of 1 completed
673 ended
Starting 674
[*********************100%***********************]  1 of 1 completed
674 ended
Starting 675
[*********************100%***********************]  1 of 1 completed
675 ended
S

...

[*********************100%***********************]  1 of 1 completed
2965 ended
Starting 2966
[*********************100%***********************]  1 of 1 completed
2966 ended
Starting 2967
[*********************100%***********************]  1 of 1 completed
2967 ended
Starting 2968
[*********************100%***********************]  1 of 1 completed
2968 ended
Starting 2969
[*********************100%***********************]  1 of 1 completed
2969 ended
Starting 2970
[*********************100%***********************]  1 of 1 completed
2970 ended
Starting 2971
[*********************100%***********************]  1 of 1 completed
2971 ended
Starting 2972
[*********************100%***********************]  1 of 1 completed
2972 ended
Starting 2973
[*********************100%***********************]  1 of 1 completed
2973 ended
Starting 2974
2974 ended
Starting 2975
2975 ended
Starting 2976
[*********************100%***********************]  1 of 1 completed
2976 ended
Starting 2977
2977 ended

[*********************100%***********************]  1 of 1 completed
3148 ended
Starting 3149
[*********************100%***********************]  1 of 1 completed
3149 ended
Starting 3150
[*********************100%***********************]  1 of 1 completed
3150 ended
Starting 3151
3151 ended
Starting 3152
[*********************100%***********************]  1 of 1 completed
3152 ended
Starting 3153
[*********************100%***********************]  1 of 1 completed
3153 ended
Starting 3154
[*********************100%***********************]  1 of 1 completed
3154 ended
Starting 3155
[*********************100%***********************]  1 of 1 completed
3155 ended
Starting 3156
[*********************100%***********************]  1 of 1 completed
3156 ended
Starting 3157
[*********************100%***********************]  1 of 1 completed
3157 ended
Starting 3158
[*********************100%***********************]  1 of 1 completed
3158 ended
Starting 3159
[*********************100%*********

[*********************100%***********************]  1 of 1 completed
3331 ended
Starting 3332
[*********************100%***********************]  1 of 1 completed
3332 ended
Starting 3333
[*********************100%***********************]  1 of 1 completed
3333 ended
Starting 3334
[*********************100%***********************]  1 of 1 completed
3334 ended
Starting 3335
[*********************100%***********************]  1 of 1 completed
3335 ended
Starting 3336
[*********************100%***********************]  1 of 1 completed
3336 ended
Starting 3337
[*********************100%***********************]  1 of 1 completed
3337 ended
Starting 3338
[*********************100%***********************]  1 of 1 completed
3338 ended
Starting 3339
[*********************100%***********************]  1 of 1 completed
3339 ended
Starting 3340
[*********************100%***********************]  1 of 1 completed
3340 ended
Starting 3341
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
3514 ended
Starting 3515
[*********************100%***********************]  1 of 1 completed
3515 ended
Starting 3516
[*********************100%***********************]  1 of 1 completed
3516 ended
Starting 3517
[*********************100%***********************]  1 of 1 completed
3517 ended
Starting 3518
[*********************100%***********************]  1 of 1 completed
3518 ended
Starting 3519
[*********************100%***********************]  1 of 1 completed
3519 ended
Starting 3520
[*********************100%***********************]  1 of 1 completed
3520 ended
Starting 3521
[*********************100%***********************]  1 of 1 completed
3521 ended
Starting 3522
[*********************100%***********************]  1 of 1 completed
3522 ended
Starting 3523
[*********************100%***********************]  1 of 1 completed
3523 ended
Starting 3524
[*********************100%***********************]  1 of 1 c

[*********************100%***********************]  1 of 1 completed
3694 ended
Starting 3695
[*********************100%***********************]  1 of 1 completed
3695 ended
Starting 3696
[*********************100%***********************]  1 of 1 completed
3696 ended
Starting 3697
[*********************100%***********************]  1 of 1 completed
3697 ended
Starting 3698
[*********************100%***********************]  1 of 1 completed
3698 ended
Starting 3699
[*********************100%***********************]  1 of 1 completed
3699 ended
Starting 3700
3700 ended
Starting 3701
[*********************100%***********************]  1 of 1 completed
3701 ended
Starting 3702
[*********************100%***********************]  1 of 1 completed
3702 ended
Starting 3703
[*********************100%***********************]  1 of 1 completed
3703 ended
Starting 3704
[*********************100%***********************]  1 of 1 completed
3704 ended
Starting 3705
3705 ended
Starting 3706
[*********

In [158]:
len(pricelist.keys())

3513876

In [24]:
pricelistdf.to_parquet('stockclose.parquet')
pricelistdf.to_csv('stockclose.csv')

The way I did it above is kind of slow. If you just call the yfinance API fewer times, it should run faster. I attempted that below, but the resulting dataframe is not efficient.

In [28]:
%%time
tickerdf = pd.DataFrame({'time_id':[], 'investment_id':[], })
ticker = []
for i in range(0,3774):
    print('Starting',i)    
    if i in mapping.keys():
        ticker.append(mapping[i])
    if i %100==0:
        price = yf.download(ticker, start='2014-01-01', end='2018-12-31')
        tickerdf = pd.concat([tickerdf, price])# = price.loc[price.index==timemap[k]].Close
        ticker = []
    print(i,"ended")


Starting 0
[*********************100%***********************]  1 of 1 completed
0 ended
Starting 1
1 ended
Starting 2
2 ended
Starting 3
3 ended
Starting 4
4 ended
Starting 5
5 ended
Starting 6
6 ended
Starting 7
7 ended
Starting 8
8 ended
Starting 9
9 ended
Starting 10
10 ended
Starting 11
11 ended
Starting 12
12 ended
Starting 13
13 ended
Starting 14
14 ended
Starting 15
15 ended
Starting 16
16 ended
Starting 17
17 ended
Starting 18
18 ended
Starting 19
19 ended
Starting 20
20 ended
Starting 21
21 ended
Starting 22
22 ended
Starting 23
23 ended
Starting 24
24 ended
Starting 25
25 ended
Starting 26
26 ended
Starting 27
27 ended
Starting 28
28 ended
Starting 29
29 ended
Starting 30
30 ended
Starting 31
31 ended
Starting 32
32 ended
Starting 33
33 ended
Starting 34
34 ended
Starting 35
35 ended
Starting 36
36 ended
Starting 37
37 ended
Starting 38
38 ended
Starting 39
39 ended
Starting 40
40 ended
Starting 41
41 ended
Starting 42
42 ended
Starting 43
43 ended
Starting 44
44 ended
Starti

[*********************100%***********************]  93 of 93 completed
400 ended
Starting 401
401 ended
Starting 402
402 ended
Starting 403
403 ended
Starting 404
404 ended
Starting 405
405 ended
Starting 406
406 ended
Starting 407
407 ended
Starting 408
408 ended
Starting 409
409 ended
Starting 410
410 ended
Starting 411
411 ended
Starting 412
412 ended
Starting 413
413 ended
Starting 414
414 ended
Starting 415
415 ended
Starting 416
416 ended
Starting 417
417 ended
Starting 418
418 ended
Starting 419
419 ended
Starting 420
420 ended
Starting 421
421 ended
Starting 422
422 ended
Starting 423
423 ended
Starting 424
424 ended
Starting 425
425 ended
Starting 426
426 ended
Starting 427
427 ended
Starting 428
428 ended
Starting 429
429 ended
Starting 430
430 ended
Starting 431
431 ended
Starting 432
432 ended
Starting 433
433 ended
Starting 434
434 ended
Starting 435
435 ended
Starting 436
436 ended
Starting 437
437 ended
Starting 438
438 ended
Starting 439
439 ended
Starting 440
440 ended

[*********************100%***********************]  94 of 94 completed
800 ended
Starting 801
801 ended
Starting 802
802 ended
Starting 803
803 ended
Starting 804
804 ended
Starting 805
805 ended
Starting 806
806 ended
Starting 807
807 ended
Starting 808
808 ended
Starting 809
809 ended
Starting 810
810 ended
Starting 811
811 ended
Starting 812
812 ended
Starting 813
813 ended
Starting 814
814 ended
Starting 815
815 ended
Starting 816
816 ended
Starting 817
817 ended
Starting 818
818 ended
Starting 819
819 ended
Starting 820
820 ended
Starting 821
821 ended
Starting 822
822 ended
Starting 823
823 ended
Starting 824
824 ended
Starting 825
825 ended
Starting 826
826 ended
Starting 827
827 ended
Starting 828
828 ended
Starting 829
829 ended
Starting 830
830 ended
Starting 831
831 ended
Starting 832
832 ended
Starting 833
833 ended
Starting 834
834 ended
Starting 835
835 ended
Starting 836
836 ended
Starting 837
837 ended
Starting 838
838 ended
Starting 839
839 ended
Starting 840
840 ended

[*********************100%***********************]  94 of 94 completed
1200 ended
Starting 1201
1201 ended
Starting 1202
1202 ended
Starting 1203
1203 ended
Starting 1204
1204 ended
Starting 1205
1205 ended
Starting 1206
1206 ended
Starting 1207
1207 ended
Starting 1208
1208 ended
Starting 1209
1209 ended
Starting 1210
1210 ended
Starting 1211
1211 ended
Starting 1212
1212 ended
Starting 1213
1213 ended
Starting 1214
1214 ended
Starting 1215
1215 ended
Starting 1216
1216 ended
Starting 1217
1217 ended
Starting 1218
1218 ended
Starting 1219
1219 ended
Starting 1220
1220 ended
Starting 1221
1221 ended
Starting 1222
1222 ended
Starting 1223
1223 ended
Starting 1224
1224 ended
Starting 1225
1225 ended
Starting 1226
1226 ended
Starting 1227
1227 ended
Starting 1228
1228 ended
Starting 1229
1229 ended
Starting 1230
1230 ended
Starting 1231
1231 ended
Starting 1232
1232 ended
Starting 1233
1233 ended
Starting 1234
1234 ended
Starting 1235
1235 ended
Starting 1236
1236 ended
Starting 1237
1237

[*********************100%***********************]  98 of 98 completed
1600 ended
Starting 1601
1601 ended
Starting 1602
1602 ended
Starting 1603
1603 ended
Starting 1604
1604 ended
Starting 1605
1605 ended
Starting 1606
1606 ended
Starting 1607
1607 ended
Starting 1608
1608 ended
Starting 1609
1609 ended
Starting 1610
1610 ended
Starting 1611
1611 ended
Starting 1612
1612 ended
Starting 1613
1613 ended
Starting 1614
1614 ended
Starting 1615
1615 ended
Starting 1616
1616 ended
Starting 1617
1617 ended
Starting 1618
1618 ended
Starting 1619
1619 ended
Starting 1620
1620 ended
Starting 1621
1621 ended
Starting 1622
1622 ended
Starting 1623
1623 ended
Starting 1624
1624 ended
Starting 1625
1625 ended
Starting 1626
1626 ended
Starting 1627
1627 ended
Starting 1628
1628 ended
Starting 1629
1629 ended
Starting 1630
1630 ended
Starting 1631
1631 ended
Starting 1632
1632 ended
Starting 1633
1633 ended
Starting 1634
1634 ended
Starting 1635
1635 ended
Starting 1636
1636 ended
Starting 1637
1637

[*********************100%***********************]  94 of 94 completed
2000 ended
Starting 2001
2001 ended
Starting 2002
2002 ended
Starting 2003
2003 ended
Starting 2004
2004 ended
Starting 2005
2005 ended
Starting 2006
2006 ended
Starting 2007
2007 ended
Starting 2008
2008 ended
Starting 2009
2009 ended
Starting 2010
2010 ended
Starting 2011
2011 ended
Starting 2012
2012 ended
Starting 2013
2013 ended
Starting 2014
2014 ended
Starting 2015
2015 ended
Starting 2016
2016 ended
Starting 2017
2017 ended
Starting 2018
2018 ended
Starting 2019
2019 ended
Starting 2020
2020 ended
Starting 2021
2021 ended
Starting 2022
2022 ended
Starting 2023
2023 ended
Starting 2024
2024 ended
Starting 2025
2025 ended
Starting 2026
2026 ended
Starting 2027
2027 ended
Starting 2028
2028 ended
Starting 2029
2029 ended
Starting 2030
2030 ended
Starting 2031
2031 ended
Starting 2032
2032 ended
Starting 2033
2033 ended
Starting 2034
2034 ended
Starting 2035
2035 ended
Starting 2036
2036 ended
Starting 2037
2037

[*********************100%***********************]  92 of 92 completed
2400 ended
Starting 2401
2401 ended
Starting 2402
2402 ended
Starting 2403
2403 ended
Starting 2404
2404 ended
Starting 2405
2405 ended
Starting 2406
2406 ended
Starting 2407
2407 ended
Starting 2408
2408 ended
Starting 2409
2409 ended
Starting 2410
2410 ended
Starting 2411
2411 ended
Starting 2412
2412 ended
Starting 2413
2413 ended
Starting 2414
2414 ended
Starting 2415
2415 ended
Starting 2416
2416 ended
Starting 2417
2417 ended
Starting 2418
2418 ended
Starting 2419
2419 ended
Starting 2420
2420 ended
Starting 2421
2421 ended
Starting 2422
2422 ended
Starting 2423
2423 ended
Starting 2424
2424 ended
Starting 2425
2425 ended
Starting 2426
2426 ended
Starting 2427
2427 ended
Starting 2428
2428 ended
Starting 2429
2429 ended
Starting 2430
2430 ended
Starting 2431
2431 ended
Starting 2432
2432 ended
Starting 2433
2433 ended
Starting 2434
2434 ended
Starting 2435
2435 ended
Starting 2436
2436 ended
Starting 2437
2437

[*********************100%***********************]  92 of 92 completed
2800 ended
Starting 2801
2801 ended
Starting 2802
2802 ended
Starting 2803
2803 ended
Starting 2804
2804 ended
Starting 2805
2805 ended
Starting 2806
2806 ended
Starting 2807
2807 ended
Starting 2808
2808 ended
Starting 2809
2809 ended
Starting 2810
2810 ended
Starting 2811
2811 ended
Starting 2812
2812 ended
Starting 2813
2813 ended
Starting 2814
2814 ended
Starting 2815
2815 ended
Starting 2816
2816 ended
Starting 2817
2817 ended
Starting 2818
2818 ended
Starting 2819
2819 ended
Starting 2820
2820 ended
Starting 2821
2821 ended
Starting 2822
2822 ended
Starting 2823
2823 ended
Starting 2824
2824 ended
Starting 2825
2825 ended
Starting 2826
2826 ended
Starting 2827
2827 ended
Starting 2828
2828 ended
Starting 2829
2829 ended
Starting 2830
2830 ended
Starting 2831
2831 ended
Starting 2832
2832 ended
Starting 2833
2833 ended
Starting 2834
2834 ended
Starting 2835
2835 ended
Starting 2836
2836 ended
Starting 2837
2837

[*********************100%***********************]  95 of 95 completed
3200 ended
Starting 3201
3201 ended
Starting 3202
3202 ended
Starting 3203
3203 ended
Starting 3204
3204 ended
Starting 3205
3205 ended
Starting 3206
3206 ended
Starting 3207
3207 ended
Starting 3208
3208 ended
Starting 3209
3209 ended
Starting 3210
3210 ended
Starting 3211
3211 ended
Starting 3212
3212 ended
Starting 3213
3213 ended
Starting 3214
3214 ended
Starting 3215
3215 ended
Starting 3216
3216 ended
Starting 3217
3217 ended
Starting 3218
3218 ended
Starting 3219
3219 ended
Starting 3220
3220 ended
Starting 3221
3221 ended
Starting 3222
3222 ended
Starting 3223
3223 ended
Starting 3224
3224 ended
Starting 3225
3225 ended
Starting 3226
3226 ended
Starting 3227
3227 ended
Starting 3228
3228 ended
Starting 3229
3229 ended
Starting 3230
3230 ended
Starting 3231
3231 ended
Starting 3232
3232 ended
Starting 3233
3233 ended
Starting 3234
3234 ended
Starting 3235
3235 ended
Starting 3236
3236 ended
Starting 3237
3237

[*********************100%***********************]  95 of 95 completed
3600 ended
Starting 3601
3601 ended
Starting 3602
3602 ended
Starting 3603
3603 ended
Starting 3604
3604 ended
Starting 3605
3605 ended
Starting 3606
3606 ended
Starting 3607
3607 ended
Starting 3608
3608 ended
Starting 3609
3609 ended
Starting 3610
3610 ended
Starting 3611
3611 ended
Starting 3612
3612 ended
Starting 3613
3613 ended
Starting 3614
3614 ended
Starting 3615
3615 ended
Starting 3616
3616 ended
Starting 3617
3617 ended
Starting 3618
3618 ended
Starting 3619
3619 ended
Starting 3620
3620 ended
Starting 3621
3621 ended
Starting 3622
3622 ended
Starting 3623
3623 ended
Starting 3624
3624 ended
Starting 3625
3625 ended
Starting 3626
3626 ended
Starting 3627
3627 ended
Starting 3628
3628 ended
Starting 3629
3629 ended
Starting 3630
3630 ended
Starting 3631
3631 ended
Starting 3632
3632 ended
Starting 3633
3633 ended
Starting 3634
3634 ended
Starting 3635
3635 ended
Starting 3636
3636 ended
Starting 3637
3637

In [29]:
price = yf.download(ticker, start='2014-01-01', end='2018-12-31')
tickerdf = pd.concat([tickerdf, price])# = price.loc[price.index==timemap[k]].Close

[*********************100%***********************]  69 of 69 completed


In [31]:
price

Adj Close                                                    \
           000032.SZ 000099.SZ 000518.SZ 000531.SZ 000564.SZ 000598.SZ   
Date                                                                     
2014-01-02  7.202123  8.250695      3.33  5.097950  4.463260  5.232164   
2014-01-03  7.120835  8.125974      3.29  4.983816  4.374781  5.304708   
2014-01-06  6.885099  7.828567      3.21  4.789367  4.217486  5.232164   
2014-01-07  6.698135  7.857347      3.22  4.814730  4.217486  5.186825   
2014-01-08  6.844455  7.799785      3.19  4.768231  4.178162  5.223097   
...              ...       ...       ...       ...       ...       ...   
2018-12-24  7.213995  5.661486      3.50  4.633865  2.640000  3.895333   
2018-12-25  7.046227  5.602818      3.50  5.099113  2.560000  3.838467   
2018-12-26  7.056096  5.602818      3.48  4.978148  2.570000  3.810033   
2018-12-27  6.898197  5.553927      3.44  4.838574  2.520000  3.810033   
2018-12-28  6.927803  5.514815      3.46  4.708305  2.530000  3.819511   

                                                    ...    Volume            \
           000650.SZ 000703.SZ 000835.SZ 000976.SZ  ... 600708.SS 600753.SS   
Date                                                ...                       
2014-01-02  3.870978  3.538388      5.38  3.370584  ...   4804691   2192302   
2014-01-03  3.799427  3.441509      5.50  3.390470  ...   9198188   1108769   
2014-01-06  3.663477  3.335403      5.44  3.281100  ...  16485956   1549502   
2014-01-07  3.706408  3.312337      5.41  3.300985  ...  11597565   1303716   
2014-01-08  3.684942  3.252364      5.44  3.251272  ...  13129819   1077995   
...              ...       ...       ...       ...  ...       ...       ...   
2018-12-24  5.775545  8.329786      4.74  4.981307  ...   3621159    747072   
2018-12-25  5.651648  8.166595      4.56  4.752624  ...   5967803   1857471   
2018-12-26  5.613525  8.152405      4.64  4.712852  ...   3970303   1036896   
2018-12-27  5.508689  8.173691      4.36  4.424513  ...   5401600   1742076   
2018-12-28  5.222771  8.173691      4.16  4.553769  ...   4698098    899095   

                                                                             \
           600804.SS  603035.SS  603078.SS  603319.SS  603707.SS  603722.SS   
Date                                                                          
2014-01-02  28276792        NaN        NaN        NaN        NaN        NaN   
2014-01-03  32829146        NaN        NaN        NaN        NaN        NaN   
2014-01-06  79819052        NaN        NaN        NaN        NaN        NaN   
2014-01-07  43790555        NaN        NaN        NaN        NaN        NaN   
2014-01-08  57022975        NaN        NaN        NaN        NaN        NaN   
...              ...        ...        ...        ...        ...        ...   
2018-12-24  12686365   885285.0   994318.0  1004094.0  2594797.0  2458806.0   
2018-12-25  18933409  2269672.0  1336479.0  1071428.0  1756407.0  1848894.0   
2018-12-26  10348564  1130922.0  1360777.0  1155700.0  1555486.0  1556707.0   
2018-12-27  13549018  1064244.0  1072839.0  1242352.0  2167777.0  2022555.0   
2018-12-28  10791349   781100.0   844878.0   934206.0  2448705.0  2050300.0   

                                  
            603776.SS  603797.SS  
Date                              
2014-01-02        NaN        NaN  
2014-01-03        NaN        NaN  
2014-01-06        NaN        NaN  
2014-01-07        NaN        NaN  
2014-01-08        NaN        NaN  
...               ...        ...  
2018-12-24  1539081.0   660128.0  
2018-12-25  2980286.0  1312220.0  
2018-12-26  1884185.0   921984.0  
2018-12-27  2147809.0  1057028.0  
2018-12-28  5658463.0   640332.0  

[1220 rows x 414 columns]